In [1]:
%matplotlib ipympl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import os  

# extract background 

In [2]:
def extract_consecutive_ones_blocks(A, B, min_len):
    consecutive_ones_blocks_B = []
    current_block_B = []

    for idx, num in enumerate(A):
        if num == 1:
            current_block_B.append(B[idx])
        elif current_block_B:
            if len(current_block_B)>min_len:
                consecutive_ones_blocks_B.append(np.array(current_block_B))
            current_block_B = []

    if current_block_B and len(current_block_B)>min_len:
        consecutive_ones_blocks_B.append(np.array(current_block_B))

    return consecutive_ones_blocks_B

In [3]:
    def read_data( hf, target_device, house_list):
        """Load data of houses
        """

        if len(house_list) == 0:
            return [], []

        else:
            aggregates = []    
            for house in house_list:
                aggregate = hf[house]['aggregate'][:]
                aggregates.append(aggregate)
            aggregates = np.concatenate(aggregates, axis=0)
            return aggregates

In [4]:
# extract activations from predictions

app = 'aggregation'
path ='app_activations/'+app+'/'
seq_len = 2
min_len = 900
no =16
for i in range(1,6):
# for i in [2]:
    if i==4:
        continue
    print('house '+str(i))
    
    train_house_list=["house"+str(i)]
    hf = h5py.File('uk_kettle.h5', 'r')
    prediction = read_data(hf, app, train_house_list)
    effective_indexes = np.zeros(len(prediction))
    for j in range(len(prediction)):
        if prediction[j]<1000:
            effective_indexes[j] = 1
    # np.save("result",effective_indexes)
    activations = extract_consecutive_ones_blocks(effective_indexes, prediction, min_len)
    print(len(activations))
    for c, k in enumerate(activations):
        file_path = os.path.join(path, 'uk' + str(i), 'a' + str(c) + '.npy') 
        dir_path = os.path.dirname(file_path)    
        if not os.path.exists(dir_path):  
            os.makedirs(dir_path)  
        np.save(path+'uk'+str(i)+'/a'+str(c)+'.npy', k)

# Synthsis validate

In [2]:
    def read_data( hf, target_device, house_list):
        """Load data of houses
        """

        if len(house_list) == 0:
            return [], []

        else:
            aggregates = []
            targets = []
            timestamps=[]
        
            for house in house_list:

                aggregate = hf[house]['aggregate'][:]
                target = hf[house][target_device][:]
                # timestamp=hf[house]['unix'][:]
                
                aggregates.append(aggregate)
                targets.append(target)
                # timestamps.append(timestamp)
            aggregates = np.concatenate(aggregates, axis=0)
            targets = np.concatenate(targets, axis=0)
            # timestamps=np.concatenate(timestamps,axis=0)
            
            return aggregates, targets#,timestamps

In [16]:
def load_data(root_dir,outlier,num_drop = 0):
    samples=[]
    for subdir, dirs, files in os.walk(root_dir):  
        
                for file in files:  
                    if file.endswith('.npy'):
                        path = os.path.join(subdir, file) 
                        waveform = np.load(path)
                        samples.append(waveform)
    return samples
                            

In [30]:
background_noise = {}
for i in [16]:
# for i in range(1,21):
    if i==14:
        continue
    path= 'path'+str(i)+'/'
    a=load_data(path,0)
    background_noise[i]=a

In [37]:
# x=np.concatenate(background_noise[16])
# y = np.zeros(len(x))
target_device='dishwasher'
train_house_list=["house16"]
hf = h5py.File('path', 'r')
(x, y) = read_data(hf, target_device, train_house_list)
df=pd.DataFrame({'dishwasher':y,'Aggregate':x})
path="data/train16.csv"
df.to_csv(path, index=False, header=True)

In [5]:
target_device='dishwasher'
train_house_list=["house20"]
hf = h5py.File('path', 'r')
(x, y) = read_data(hf, target_device, train_house_list)
y=np.zeros(len(x))
df=pd.DataFrame({'dishwasher':y,'Aggregate':x})
path="data/validate20.csv"
df.to_csv(path, index=False, header=True)